## Import stuff

In [6]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import random
from PIL import Image
import logging
import time
import sklearn.metrics as metrics


## Load training data from csv


In [7]:
val_data = pd.read_csv("val_distances.csv",index_col=0)
val_labels = pd.read_csv("val_labels.txt", header=None)

for extractor in ["xl","resnet","clip","vit"]:
    for i in range(1,4):
        val_data[extractor+'_'+str(i)] = val_data["anchor_left_distance_"+extractor+'_'+str(i)]-val_data["anchor_right_distance_"+extractor+'_'+str(i)]
        val_data.drop("anchor_left_distance_"+extractor+'_'+str(i), axis=1, inplace=True)
        val_data.drop("anchor_right_distance_"+extractor+'_'+str(i), axis=1, inplace=True)


X_val = np.array(val_data.iloc[:, :].values,  dtype=float)
Y_val = np.array(val_labels.iloc[:, :].values,  dtype=float).reshape(-1)

print("X shape: ", X_val.shape)
print("Y shape: ", Y_val.shape)

val_data

X shape:  (7471, 12)
Y shape:  (7471,)


,xl_1,xl_2,xl_3,resnet_1,resnet_2,resnet_3,clip_1,clip_2,clip_3,vit_1,vit_2,vit_3
0,-0.322427,-0.350006,-0.408311,-0.474757,-0.470037,-0.568101,-0.445364,-0.510079,-0.553964,-0.397028,-0.283282,-0.429616
1,-0.142240,-0.115177,-0.127701,-0.141702,-0.067235,-0.134712,-0.160547,-0.120491,-0.093794,-0.105658,-0.023801,0.035956
2,0.010238,0.125666,0.059033,0.091578,0.070494,0.142473,-0.107461,-0.118825,-0.164650,0.004763,-0.126344,-0.035216
3,-0.569890,-0.685985,-0.639338,-0.648733,-0.668245,-0.601180,-1.094588,-0.829342,-0.856445,-1.149241,-1.038974,-1.183547
4,0.399750,0.567964,0.464068,0.367554,0.418747,0.442058,0.340783,0.567723,0.496654,-0.112918,-0.126657,-0.116630
...,...,...,...,...,...,...,...,...,...,...,...,...
7466,-0.530465,-0.427475,-0.370071,-0.262559,-0.203195,-0.280204,-0.985189,-0.743111,-0.688317,-0.645051,-0.743211,-0.647058
7467,-0.124510,-0.034409,-0.016012,0.229187,0.142706,0.187024,-0.053910,0.052951,-0.040656,0.128241,0.155499,0.056485
7468,-0.098082,-0.212675,-0.213816,-0.503593,-0.617256,-0.548742,0.071487,0.207409,0.178167,-0.135269,-0.286125,-0.302154
7469,0.393884,0.430400,0.369271,0.444265,0.546835,0.428938,0.521176,0.442475,0.541795,0.262059,0.299697,0.239632


# weighted ensemble
the weights would be kinda hard to estimate in practice, it would take a few server submissions. I just made the worse 2 models less important

In [18]:
pred = []
for row in X_val:
    # score_left = (row[0]+row[1]+row[2])/6 + (row[3]+row[4]+row[5])/3 + (row[6]+row[7]+row[8])/3 + (row[9]+row[10]+row[11])/6
    score_left = (row[6]+row[7]+row[8])/3
    pred.append(score_left>0)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred)
accuracy

0.7600053540356043

# Majority vote
performs worse

In [17]:
pred = []
for row in X_val:
    score_left = 0
    for i in range(12):
        score_left += row[i]>0

    pred.append(0 if score_left>6 else 1)

pred = np.array(pred)
accuracy = metrics.accuracy_score(Y_val, pred<0.5)
accuracy

0.7683041092223263

In [10]:
# val_data = pd.read_csv("val_distances.csv",index_col=0)
# val_data
# val_data.drop("anchor_left_distance_swin",inplace=True,axis=1)
# val_data.drop("anchor_right_distance_swin",inplace=True,axis=1)
# val_data.to_csv("val_distances.csv")